In [46]:
error = lambda p, p_approx: abs((p - p_approx) / p)

In [47]:
import math


def tr(num, digits):
    if num == 0:
        return 0

    # Calculate the order of magnitude of the number
    magnitude = int(math.floor(math.log10(abs(num))))

    # Calculate the scaling factor based on the desired number of significant digits
    scaling_factor = 10 ** (digits - magnitude - 1)

    # Cut the number to the desired number of significant digits without rounding
    cut_num = math.trunc(num * scaling_factor) / scaling_factor

    return cut_num


def rd(number, significant_digits):
    if number == 0:
        return 0

    magnitude = math.floor(math.log10(abs(number))) + 1
    scale = significant_digits - magnitude

    return round(number, scale)


def pls_f(a, b, digits, fn):
    return fn(fn(a, digits) + fn(b, digits), digits)


def mns_f(a, b, digits, fn):
    return fn(fn(a, digits) - fn(b, digits), digits)


def mlt_f(a, b, digits, fn):
    return fn(fn(a, digits) * fn(b, digits), digits)


def dvd_f(a, b, digits, fn):
    return fn(fn(a, digits) / fn(b, digits), digits)

In [48]:
digits = 3
fc = rd
ml = lambda a, b: mlt_f(a, b, digits, fc)
dv = lambda a, b: dvd_f(a, b, digits, fc)
pl = lambda a, b: pls_f(a, b, digits, fc)
mn = lambda a, b: mns_f(a, b, digits, fc)

x0 = 1.31
y0 = 3.24

x1 = 1.93
y1 = 4.76

x_real = (x0 * y1 - x1 * y0) / (y1 - y0)
print("real =\t", x_real)
# Caso a
a = ml(x0, y1)
b = ml(x1, y0)
c = mn(a, b)
d = mn(y1, y0)
x_a = dv(c, d)
print("x metodo 1 = ", x_a)
error_a = error(x_real, x_a)
print("error m1=", error_a)

# Caso b
a1 = mn(x1, x0)
d1 = mn(y1, y0)
c1 = ml(a1, y0)
d1 = dv(c1, d1)
x_b = x0 - d1
print("x met 2 =", x_b)
error_b = error(x_real, x_b)
print("error m2 =", error_b)

real =	 -0.011578947368421534
x metodo 1 =  -0.00658
error m1= 0.43172727272729633
x met 2 = -0.010000000000000009
error m2 = 0.1363636363636715


In [49]:
from math import pi

pi = 3.14159
atan = lambda x: x - 1 / 3 * x**3 + 1 / 5 * x**5

a = 4 * (atan(1 / 2) + atan(1 / 3))
# a = rd(a, 6)
error_a = error(pi, a)
error_a = rd(error_a, 4)
print("approx", a, "error", error_a)
b = 16 * atan(1 / 5) - 4 * atan(1 / 239)
# b = rd(b, 6)
error_b = error(pi, b)
error_b = rd(error_b, 4)
print("approx", b, "error", error_b)

approx 3.1455761316872426 error 0.001269
approx 3.1416210293250346 error 9.877e-06


# Soluciones de ecuaciones


In [50]:
def sign(x: float) -> int:
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

In [51]:
from typing import Callable


def bisection(
    a: float, b: float, *, equation: Callable[[float], float], tol: float, N: int
) -> tuple[float, float, float, int] | None:
    i = 1

    # Input validation
    # Comente a continuación y vea qué sucede
    assert a < b, "a not lower than b, the interval is not valid."

    # Comente a continuación y vea qué sucede
    assert (
        equation(a) * equation(b) < 0
    ), "The function does not change sign over the interval."

    Fa = equation(a)
    p = a  # to avoid crash in i == 0.
    for i in range(N):

        # esta alternativa es suceptible de desborde, especialmente si a y b son muy grandes
        # p = (a + b)/2

        p = a + (b - a) / 2
        FP = equation(p)
        if FP == 0 or (b - a) / 2 < tol:
            return p, a, b, i

        if sign(Fa) * sign(FP) > 0:

            # esta alternativa podría causar también un desborde
            # if Fa * FP > 0:

            a = p
            Fa = FP
        else:
            b = p

    # Respuesta temporal
    return p, a, b, i

    # Es mejor usar esta opción, cuando el algoritmo no llega a una tolerancia en
    # las iteraciones dadas.

    # raise ValueError(
    #     "Solution not found within the given tolerance and number of iterations"
    # )

In [52]:
from math import sin

bisection(-3.5, 3, equation=sin, tol=1e-6, N=1000)

AssertionError: The function does not change sign over the interval.

In [ ]:
bisection(-1, 2, equation=sin, tol=1e-6, N=1000)

(-2.384185791015625e-07, -9.5367431640625e-07, 4.76837158203125e-07, 21)

In [ ]:
bisection(-3.5, 3.5, equation=sin, tol=1e-6, N=1000)

(0.0, -3.5, 3.5, 0)

In [ ]:
bisection(-5, 4, equation=sin, tol=1e-6, N=1000)

(-3.14159232378006, -3.141592860221863, -3.141591787338257, 23)

## Método de la Secante

In [54]:
def secant_method(f, x0, x1, tol=1e-6, max_iter=100):
    """
    Secant method for finding the root of a function.

    Parameters:
    - f: The function for which to find the root.
    - x0, x1: Initial guesses for the root.
    - tol: Tolerance for convergence (default: 1e-6).
    - max_iter: Maximum number of iterations (default: 100).

    Returns:
    - The approximate root of the function.
    - The number of iterations taken.
    """

    # Initialize variables
    x_prev = x0
    x_curr = x1
    iter_count = 0

    while abs(f(x_curr)) > tol and iter_count < max_iter:
        # Calculate the next approximation using the secant method formula
        x_next = x_curr - f(x_curr) * (x_curr - x_prev) / (f(x_curr) - f(x_prev))

        # Update variables for the next iteration
        x_prev = x_curr
        x_curr = x_next
        iter_count += 1

    return x_curr, iter_count

In [55]:
from scipy.optimize import newton

i = 0


def func(x):
    global i
    i += 1
    y = x**3 - 3 * x**2 + x - 1
    print(f"Llamada i={i}\t x={x:.2f}\t y={y:.2f}")
    return y


newton(func, x0=2, x1=3)

Llamada i=1	 x=2.00	 y=-3.00
Llamada i=2	 x=3.00	 y=2.00
Llamada i=3	 x=2.60	 y=-1.10
Llamada i=4	 x=2.95	 y=1.51
Llamada i=5	 x=2.75	 y=-0.16
Llamada i=6	 x=2.77	 y=-0.02
Llamada i=7	 x=2.77	 y=0.00
Llamada i=8	 x=2.77	 y=-0.00
Llamada i=9	 x=2.77	 y=-0.00


2.769292354238631

In [63]:
i = 0


def func(x):
    global i
    i += 1
    y = x**3 - 3 * x**2 + x - 1
    print(f"Llamada i={i}\t x={x:.2f}\t y={y:.2f}")
    return y


secant_method(func, x0=2, x1=3)

Llamada i=1	 x=3.00	 y=2.00
Llamada i=2	 x=3.00	 y=2.00
Llamada i=3	 x=3.00	 y=2.00
Llamada i=4	 x=2.00	 y=-3.00
Llamada i=5	 x=2.60	 y=-1.10
Llamada i=6	 x=2.60	 y=-1.10
Llamada i=7	 x=2.60	 y=-1.10
Llamada i=8	 x=3.00	 y=2.00
Llamada i=9	 x=2.74	 y=-0.20
Llamada i=10	 x=2.74	 y=-0.20
Llamada i=11	 x=2.74	 y=-0.20
Llamada i=12	 x=2.60	 y=-1.10
Llamada i=13	 x=2.77	 y=0.03
Llamada i=14	 x=2.77	 y=0.03
Llamada i=15	 x=2.77	 y=0.03
Llamada i=16	 x=2.74	 y=-0.20
Llamada i=17	 x=2.77	 y=-0.00
Llamada i=18	 x=2.77	 y=-0.00
Llamada i=19	 x=2.77	 y=-0.00
Llamada i=20	 x=2.77	 y=0.03
Llamada i=21	 x=2.77	 y=-0.00
Llamada i=22	 x=2.77	 y=-0.00
Llamada i=23	 x=2.77	 y=-0.00
Llamada i=24	 x=2.77	 y=-0.00
Llamada i=25	 x=2.77	 y=0.00


(2.7692923542484045, 6)

In [57]:
secant_method(func, x0=3, x1=2)

Llamada i=26	 x=2.00	 y=-3.00
Llamada i=27	 x=2.00	 y=-3.00
Llamada i=28	 x=2.00	 y=-3.00
Llamada i=29	 x=3.00	 y=2.00
Llamada i=30	 x=2.60	 y=-1.10
Llamada i=31	 x=2.60	 y=-1.10
Llamada i=32	 x=2.60	 y=-1.10
Llamada i=33	 x=2.00	 y=-3.00
Llamada i=34	 x=2.95	 y=1.51
Llamada i=35	 x=2.95	 y=1.51
Llamada i=36	 x=2.95	 y=1.51
Llamada i=37	 x=2.60	 y=-1.10
Llamada i=38	 x=2.75	 y=-0.16
Llamada i=39	 x=2.75	 y=-0.16
Llamada i=40	 x=2.75	 y=-0.16
Llamada i=41	 x=2.95	 y=1.51
Llamada i=42	 x=2.77	 y=-0.02
Llamada i=43	 x=2.77	 y=-0.02
Llamada i=44	 x=2.77	 y=-0.02
Llamada i=45	 x=2.75	 y=-0.16
Llamada i=46	 x=2.77	 y=0.00
Llamada i=47	 x=2.77	 y=0.00
Llamada i=48	 x=2.77	 y=0.00
Llamada i=49	 x=2.77	 y=-0.02
Llamada i=50	 x=2.77	 y=-0.00


(2.769292278557985, 6)

# Programación

In [26]:
def secant_method_scipy(f, x0, x1, tol=1e-6, max_iter=100):
    """
    Secant method for finding the root of a function.

    Parameters:
    - f: The function for which to find the root.
    - x0, x1: Initial guesses for the root.
    - tol: Tolerance for convergence (default: 1e-6).
    - max_iter: Maximum number of iterations (default: 100).

    Returns:
    - The approximate root of the function.
    - The number of iterations taken.
    """

    # Initialize variables
    x_prev = x0
    x_curr = x1
    iter_count = 0

    while abs(f(x_curr)) > tol and iter_count < max_iter:
        # Calculate the next approximation using the secant method formula
        y_curr = f(x_curr)
        y_prev = f(x_prev)

        if abs(y_prev) < abs(y_curr):
            x_prev, x_curr = x_curr, x_prev
            y_prev, y_curr = y_curr, y_prev

        x_next = x_curr - y_curr * (x_curr - x_prev) / (y_curr - y_prev)

        # Update variables for the next iteration
        x_prev = x_curr
        x_curr = x_next
        iter_count += 1

    return x_curr, iter_count

In [28]:
def func2(x):
    y = x**3 - 3 * x**2 + x - 1
    print(f"x={x:.2f}\t y={y:.2f}")
    return y


secant_method_scipy(func2, x0=3, x1=2)

x=2.00	 y=-3.00
x=2.00	 y=-3.00
x=3.00	 y=2.00
x=2.60	 y=-1.10
x=2.60	 y=-1.10
x=3.00	 y=2.00
x=2.74	 y=-0.20
x=2.74	 y=-0.20
x=2.60	 y=-1.10
x=2.77	 y=0.03
x=2.77	 y=0.03
x=2.74	 y=-0.20
x=2.77	 y=-0.00
x=2.77	 y=-0.00
x=2.77	 y=0.03
x=2.77	 y=-0.00
x=2.77	 y=-0.00
x=2.77	 y=-0.00
x=2.77	 y=0.00


(2.7692923542484045, 6)

In [29]:
secant_method_scipy(func2, x0=2, x1=3)

x=3.00	 y=2.00
x=3.00	 y=2.00
x=2.00	 y=-3.00
x=2.60	 y=-1.10
x=2.60	 y=-1.10
x=3.00	 y=2.00
x=2.74	 y=-0.20
x=2.74	 y=-0.20
x=2.60	 y=-1.10
x=2.77	 y=0.03
x=2.77	 y=0.03
x=2.74	 y=-0.20
x=2.77	 y=-0.00
x=2.77	 y=-0.00
x=2.77	 y=0.03
x=2.77	 y=-0.00
x=2.77	 y=-0.00
x=2.77	 y=-0.00
x=2.77	 y=0.00


(2.7692923542484045, 6)

In [5]:
from scipy.optimize import newton

newton(func2, x0=2, full_output=True, x1=3)

x=2.00	 y=-3.00
x=3.00	 y=2.00
x=2.60	 y=-1.10
x=2.95	 y=1.51
x=2.75	 y=-0.16
x=2.77	 y=-0.02
x=2.77	 y=0.00
x=2.77	 y=-0.00
x=2.77	 y=-0.00


(2.769292354238631,
       converged: True
            flag: converged
  function_calls: 9
      iterations: 8
            root: 2.769292354238631
          method: secant)

In [40]:
def secant_method_opt(f, x0, x1, tol=1e-6, max_iter=100):
    """
    Secant method for finding the root of a function.

    Parameters:
    - f: The function for which to find the root.
    - x0, x1: Initial guesses for the root.
    - tol: Tolerance for convergence (default: 1e-6).
    - max_iter: Maximum number of iterations (default: 100).

    Returns:
    - The approximate root of the function.
    - The number of iterations taken.
    """

    # Initialize variables
    x_prev = x0
    x_curr = x1
    iter_count = 0
    y_prev = f(x_prev)
    y_curr = f(x_curr)
    while abs(y_curr) > tol and iter_count < max_iter:
        # Calculate the next approximation using the secant method formula
        x_next = x_curr - y_curr * (x_curr - x_prev) / (y_curr - y_prev)

        # Update variables for the next iteration
        x_prev = x_curr
        x_curr = x_next
        y_prev = y_curr
        y_curr = f(x_curr)

        iter_count += 1

    return x_curr, iter_count

In [41]:
def func2(x):
    y = x**3 - 3 * x**2 + x - 1
    print(f"x={x:.2f}\t y={y:.2f}")
    return y


secant_method_opt(func2, x0=3, x1=2, tol=1e-3)

x=3.00	 y=2.00
x=2.00	 y=-3.00
x=2.60	 y=-1.10
x=2.95	 y=1.51
x=2.75	 y=-0.16
x=2.77	 y=-0.02
x=2.77	 y=0.00


(2.769333095055652, 5)

# Newton

In [84]:
f_prime = lambda x: 3 * x**2 - 6 * x + 1


def func2(x):
    y = x**3 - 3 * x**2 + x - 1
    print(f"x={x:.2f}\t y={y:.2f}")
    return y


# newton(func2, x0=3, fprime=f_prime, full_output=True)
# newton(func2, x0=1, fprime=f_prime, full_output=True)
# newton(func2, x0=0, fprime=f_prime, full_output=True)
newton(func2, x0=1 + 6 ** (1 / 2) / 3, fprime=f_prime, full_output=True)

x=1.82	 y=-3.09


RuntimeError: Derivative was zero. Failed to converge after 1 iterations, value is 1.8164965809277258.

In [ ]:
from scipy.optimize import newton


# ####################################################################
def f(x: float) -> float:
    y = x**3 - 3 * x**2 - 2
    print(f"x={x:.3f}\t y={y:.2f}")
    return y


def f_prime(x: float) -> float:
    return 3 * x**2 - 6 * x


newton(f, x0=3, fprime=f_prime, full_output=True)

x=3.000	 y=-2.00
x=3.222	 y=0.31
x=3.196	 y=0.00
x=3.196	 y=0.00
x=3.196	 y=0.00


(3.195823345445647,
       converged: True
            flag: converged
  function_calls: 10
      iterations: 5
            root: 3.195823345445647
          method: newton)

In [ ]:
newton(f, x0=2, fprime=f_prime, full_output=True)

x=2.000	 y=-6.00


RuntimeError: Derivative was zero. Failed to converge after 1 iterations, value is 2.0.

In [ ]:
newton(f, x0=0, fprime=f_prime, full_output=True)

x=0.000	 y=-2.00


RuntimeError: Derivative was zero. Failed to converge after 1 iterations, value is 0.0.

In [ ]:
newton(f, x0=-2, fprime=f_prime, full_output=True)

x=-2.000	 y=-22.00
x=-1.083	 y=-6.79
x=-0.406	 y=-2.56
x=0.469	 y=-2.56
x=-0.717	 y=-3.91
x=-0.048	 y=-2.01
x=6.709	 y=164.96
x=4.969	 y=46.61
x=3.916	 y=12.04
x=3.381	 y=2.35
x=3.213	 y=0.20
x=3.196	 y=0.00
x=3.196	 y=0.00
x=3.196	 y=0.00


(3.1958233454456466,
       converged: True
            flag: converged
  function_calls: 28
      iterations: 14
            root: 3.1958233454456466
          method: newton)

# Interpolación

In [ ]:
newton(f, x0=0, fprime=f_prime, full_output=True)

x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00
x=0.00	 y=-1.00
x=1.00	 y=-2.00


RuntimeError: Failed to converge after 50 iterations, value is 0.0.

In [ ]:
newton(f, x0=1.8165, fprime=f_prime, full_output=True)

x=1.82	 y=-3.09
x=184399.55	 y=6270072108749772.00
x=122933.37	 y=1857799143305945.75
x=81955.91	 y=550459005405771.50
x=54637.61	 y=163098964552530.97
x=36425.41	 y=48325619118581.03
x=24283.94	 y=14318701955664.32
x=16189.62	 y=4242578353635.88
x=10793.42	 y=1257060250530.32
x=7195.94	 y=372462294854.03
x=4797.63	 y=110359197408.83
x=3198.75	 y=32699020743.34
x=2132.84	 y=9688598264.51
x=1422.22	 y=2870695465.73
x=948.48	 y=850576223.22
x=632.66	 y=252022443.77
x=422.10	 y=74673222.58
x=281.74	 y=22125336.38
x=188.16	 y=6555613.11
x=125.78	 y=1942375.64
x=84.19	 y=575499.70
x=56.46	 y=170505.59
x=37.98	 y=50511.44
x=25.67	 y=14960.35
x=17.46	 y=4428.52
x=12.01	 y=1309.19
x=8.38	 y=385.76
x=6.00	 y=112.68
x=4.45	 y=32.14
x=3.50	 y=8.55
x=2.98	 y=1.83
x=2.80	 y=0.20
x=2.77	 y=0.00
x=2.77	 y=0.00
x=2.77	 y=0.00


(2.7692923542386314,
       converged: True
            flag: converged
  function_calls: 70
      iterations: 35
            root: 2.7692923542386314
          method: newton)

In [ ]:
import sympy as sym


def lagrange_approx(*, xs: list[float], ys: list[float]) -> sym.Symbol:
    """Approximate a set of points using Lagrange polynomials.

    ## Parameters
    ``x``: list of x values
    ``y``: list of y values

    ## Return
    ``lagrange``: the Lagrange polynomial

    """

    x = sym.symbols("x")
    lagrange: sym.Symbol = sym.S.Zero

    # -- Summation of the Lagrange polynomials
    for i in range(len(xs)):

        # -- Calculation of the Lagrange polynomial
        term = sym.S.One
        for j in range(len(xs)):
            if i != j:
                term *= (x - xs[j]) / (xs[i] - xs[j])
        term = sym.expand(term)
        lagrange += term * ys[i]

    return lagrange

In [ ]:
lagrange_approx(xs=[-1, 0, 1, 2], ys=[-1, 0, 1, 2])

x

# Cubic splines

In [ ]:
import sympy as sym
from IPython.display import display


# ###################################################################
def cubic_spline(xs: list[float], ys: list[float]) -> list[sym.Symbol]:
    """
    Cubic spline interpolation ``S``. Every two points are interpolated by a cubic polynomial
    ``S_j`` of the form ``S_j(x) = a_j + b_j(x - x_j) + c_j(x - x_j)^2 + d_j(x - x_j)^3.``

    xs must be different  but not necessarily ordered nor equally spaced.

    ## Parameters
    - xs, ys: points to be interpolated

    ## Return
    - List of symbolic expressions for the cubic spline interpolation.
    """

    points = sorted(zip(xs, ys), key=lambda x: x[0])  # sort points by x

    xs = [x for x, _ in points]
    ys = [y for _, y in points]

    n = len(points) - 1  # number of splines

    h = [xs[i + 1] - xs[i] for i in range(n)]  # distances between  contiguous xs

    alpha = [0] * n
    for i in range(1, n):
        alpha[i] = 3 / h[i] * (ys[i + 1] - ys[i]) - 3 / h[i - 1] * (ys[i] - ys[i - 1])

    l = [1]
    u = [0]
    z = [0]

    for i in range(1, n):
        l += [2 * (xs[i + 1] - xs[i - 1]) - h[i - 1] * u[i - 1]]
        u += [h[i] / l[i]]
        z += [(alpha[i] - h[i - 1] * z[i - 1]) / l[i]]

    l.append(1)
    z.append(0)
    c = [0] * (n + 1)

    x = sym.Symbol("x")
    splines = []
    for j in range(n - 1, -1, -1):
        c[j] = z[j] - u[j] * c[j + 1]
        b = (ys[j + 1] - ys[j]) / h[j] - h[j] * (c[j + 1] + 2 * c[j]) / 3
        d = (c[j + 1] - c[j]) / (3 * h[j])
        a = ys[j]
        print(j, a, b, c[j], d)
        S = a + b * (x - xs[j]) + c[j] * (x - xs[j]) ** 2 + d * (x - xs[j]) ** 3

        splines.append(S)
    splines.reverse()
    return splines

In [ ]:
import sympy as sym
from IPython.display import display


# ###################################################################
def cubic_spline_clamped(
    xs: list[float], ys: list[float], d0: float, dn: float
) -> list[sym.Symbol]:
    """
    Cubic spline interpolation ``S``. Every two points are interpolated by a cubic polynomial
    ``S_j`` of the form ``S_j(x) = a_j + b_j(x - x_j) + c_j(x - x_j)^2 + d_j(x - x_j)^3.``

    xs must be different  but not necessarily ordered nor equally spaced.

    ## Parameters
    - xs, ys: points to be interpolated
    - d0, dn: derivatives at the first and last points

    ## Return
    - List of symbolic expressions for the cubic spline interpolation.
    """

    points = sorted(zip(xs, ys), key=lambda x: x[0])  # sort points by x
    xs = [x for x, _ in points]
    ys = [y for _, y in points]
    n = len(points) - 1  # number of splines
    h = [xs[i + 1] - xs[i] for i in range(n)]  # distances between  contiguous xs

    alpha = [0] * (n + 1)  # prealloc
    alpha[0] = 3 / h[0] * (ys[1] - ys[0]) - 3 * d0
    alpha[-1] = 3 * dn - 3 / h[n - 1] * (ys[n] - ys[n - 1])

    for i in range(1, n):
        alpha[i] = 3 / h[i] * (ys[i + 1] - ys[i]) - 3 / h[i - 1] * (ys[i] - ys[i - 1])

    l = [2 * h[0]]
    u = [0.5]
    z = [alpha[0] / l[0]]

    for i in range(1, n):
        l += [2 * (xs[i + 1] - xs[i - 1]) - h[i - 1] * u[i - 1]]
        u += [h[i] / l[i]]
        z += [(alpha[i] - h[i - 1] * z[i - 1]) / l[i]]

    l.append(h[n - 1] * (2 - u[n - 1]))
    z.append((alpha[n] - h[n - 1] * z[n - 1]) / l[n])
    c = [0] * (n + 1)  # prealloc
    c[-1] = z[-1]

    x = sym.Symbol("x")
    splines = []
    for j in range(n - 1, -1, -1):
        c[j] = z[j] - u[j] * c[j + 1]
        b = (ys[j + 1] - ys[j]) / h[j] - h[j] * (c[j + 1] + 2 * c[j]) / 3
        d = (c[j + 1] - c[j]) / (3 * h[j])
        a = ys[j]
        print(j, a, b, c[j], d)
        S = a + b * (x - xs[j]) + c[j] * (x - xs[j]) ** 2 + d * (x - xs[j]) ** 3

        splines.append(S)
    splines.reverse()
    return splines

In [ ]:
xs = [
    -1,
    1,
]
ys = [
    1,
    3,
]

ss = cubic_spline_clamped(xs=xs, ys=ys, d0=1, dn=2)
_ = [display(s) for s in ss]
print("______")
_ = [display(s.expand()) for s in ss]

0 1 1.0 -0.5 0.25


1.0*x + 0.25*(x + 1)**3 - 0.5*(x + 1)**2 + 2.0

______


0.25*x**3 + 0.25*x**2 + 0.75*x + 1.75

In [ ]:
xs = [-1, 0, 1, 2]
ys = [3, 1, 5, 3]

ss = cubic_spline(xs=xs, ys=ys)
_ = [display(s) for s in ss]

2 5 2.0 -6.0 2.0
1 1 2.0 6.0 -4.0
0 3 -4.0 0.0 2.0


-4.0*x + 2.0*(x + 1)**3 - 1.0

-4.0*x**3 + 6.0*x**2 + 2.0*x + 1

2.0*x + 2.0*(x - 1)**3 - 6.0*(x - 1)**2 + 3.0